# Streaming process

In [demo_pipeline.ipynb](demo_pipeline.ipynb) we showcase the use of the `EvaluatorBuilder`
and `EvaluatorPipeline` to create a pipeline that can be used to evaluate 
the algorithm on a dataset in a particular setting.

In the event that the programmer wants to decouple the evaluation platform from
the algorithm, that is the algorithm to be instantiated in a seperate process
or the algorithm is some online algorithm that can be called on the fly. Then   
the programmer can make use of the `EvaluatorStream` class to create a streaming
process that can be used to evaluate the algorithm. This removes the burden of
instantiating and error handling of the algorithm by the evaluation platform.

Similarly to the use of the pipeline, we recommend the programmer to use `EvaluatorStreamerBuilder`
to build the streaming object.

In [1]:
k = 10

In [ ]:
from streamsightv2.datasets import TestDataset
from streamsightv2.settings import SlidingWindowSetting
dataset = TestDataset()
data = dataset.load()
setting_window = SlidingWindowSetting(
    4,
    3,
    1,
    k
)
setting_window.split(data)

INFO     - streamsight.datasets.base - TestDataset is loading dataset...
INFO     - streamsight.datasets.base - TestDataset dataset loaded - Took 0.00222s


3it [00:00, 460.05it/s]              

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 3 seconds. Number of splits: 3 in total.


In [ ]:
from streamsightv2.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
evaluator = builder.build()

## Registration of algorithm

Below we demonstrate how to register an algorithm with the evaluator. The
algorithm should ideally inherit from the BaseAlgorithm class.

In the event that the algorithm does not inherit from the BaseAlgorithm class,
it can be the case that the algorithm might return predictions that are unexpected
and error messages might be raised. We also note that the expected data communication
is through InterationMatrix objects. Thus, the provided algorithm should be able to
handle these objects.

In [ ]:
from streamsightv2.algorithms import ItemKNNIncremental

algo = ItemKNNIncremental(K=10)
algo_id = evaluator.register_algorithm(algo)
print(algo_id)

from streamsightv2.algorithms import ItemKNNStatic
external_model = ItemKNNIncremental(K=10)
external_model_id = evaluator.register_algorithm(external_model)
print(external_model_id)

INFO     - streamsight.evaluators.evaluator_stream - Registering algorithm name None with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
bdd640fb-0667-4ad1-9c80-317fa3b1799d
INFO     - streamsight.evaluators.evaluator_stream - Registering algorithm name None with ID: 23b8c1e9-3924-46de-beb1-3b9046685257
23b8c1e9-3924-46de-beb1-3b9046685257


Once the evaluator is created, we can start the evaluation process by calling
the `start_stream` method. This will signal the start of the evaluation process
and stop all registration for algorithms.

In [5]:
evaluator.start_stream()

In [6]:
evaluator.get_all_algorithm_status()

{'ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.NEW: 'NEW'>,
 'ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257': <AlgorithmStateEnum.NEW: 'NEW'>}

## Negative example 1

Here we demonstrate an example of what happens when the external model or programmer
calls the API in an out of order fashion. This is not recommended and will result in
a warning message. 

We expect the cycle to be as follows:
1. Register algorithm
2. Request for training data
3. Request for unlabeled data
4. Submit predictions
6. Repeat 2-4 until the end of the stream

In [7]:
evaluator.get_unlabeled_data(algo_id)

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/evaluators/evaluator_stream.py:296: AlgorithmStatusWarning: Algorithm:bdd640fb-0667-4ad1-9c80-317fa3b1799d current status is NEW.
  warn(AlgorithmStatusWarning(algo_id, status, "unlabeled"))



The expected call would be to get the training data first.

Note how the status of the algorithm changes on the end of the evaluator platform.
The status of the algorithm is updated after each API call to ensure that the
algorithm do not receive some data in the future and to ensure that evaluation
is in the correct window.

In [8]:
data = evaluator.get_data(algo_id)
algo.fit(data)

evaluator.get_all_algorithm_status()

{'ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.READY: 'READY'>,
 'ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257': <AlgorithmStateEnum.NEW: 'NEW'>}

## Negative example 2

Future calls that do not adhere to the expected loop will cause warning messages
to be raised, but no exceptions will be thrown. This will allow the programmer
to continue executing the code without any interruptions.

In [9]:
evaluator.get_data(algo_id)

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/evaluators/evaluator_stream.py:249: AlgorithmStatusWarning: Algorithm:bdd640fb-0667-4ad1-9c80-317fa3b1799d current status is READY.
  warn(AlgorithmStatusWarning(algo_id, status, "data_release"))



   interactionid  uid  iid  ts
0              0    0    0   0
1              1    1    0   1
2              2    2    1   2
3              3    0    2   3

In [10]:
unlabeled_data = evaluator.get_unlabeled_data(algo_id)
evaluator.get_all_algorithm_status()

{'ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.READY: 'READY'>,
 'ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257': <AlgorithmStateEnum.NEW: 'NEW'>}

In [11]:
prediction = algo.predict(unlabeled_data)
evaluator.submit_prediction(algo_id, prediction)

evaluator.get_all_algorithm_status()

{'ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.PREDICTED: 'PREDICTED'>,
 'ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257': <AlgorithmStateEnum.NEW: 'NEW'>}

## Stream to next window

When the algorithm is done predicting for the current window, the evaluator will
check if all other algorithms are done predicting for the current window. If
they are, the evaluator will stream to the next window.

Else, the evaluator will wait for the other algorithms to finish predicting and 
a warning message will be displayed for the current algorithm to wait for the
other algorithms to finish predicting. At the moment there is no multi-threading
implemented so the algorithms will have to be executed in step.

In [12]:
data = evaluator.get_data(algo_id)

INFO     - streamsight.evaluators.evaluator_stream - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d has already predicted for this data segment, please wait for all other algorithms to predict
Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d has already predicted for this data segment, please wait for all other algorithms to predict


## Positive example

The following API calls are the expected API calls for a positive example. Ideally
the API calls should be in the same order as the code snippet below.

While it may not always be possible to adhere to the following call order, the
algorithm or programmer who is using streamsight should keep track of the warning
messages to ensure that the algorithm does not infinitely wait for data or
loop in a cycle.

In [13]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
unlabeled_data = evaluator.get_unlabeled_data(external_model_id)

prediction = external_model.predict(unlabeled_data)
evaluator.submit_prediction(external_model_id, prediction)

In [14]:
evaluator.get_all_algorithm_status()

{'ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.PREDICTED: 'PREDICTED'>,
 'ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257': <AlgorithmStateEnum.PREDICTED: 'PREDICTED'>}

In [15]:
data = evaluator.get_data(algo_id)

In [16]:
evaluator.metric_results("micro")

,,micro_score,num_user
Algorithm,Metric,,
ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257,PrecisionK_10,0.1,1
ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d,PrecisionK_10,0.1,1


In [17]:
evaluator.metric_results("macro")

,,macro_score,num_window
Algorithm,Metric,,
ItemKNNIncremental(K=10)_23b8c1e9-3924-46de-beb1-3b9046685257,PrecisionK_10,0.1,1
ItemKNNIncremental(K=10)_bdd640fb-0667-4ad1-9c80-317fa3b1799d,PrecisionK_10,0.1,1
